<a href="https://colab.research.google.com/github/mohammadRahimi1993/PythonEducation/blob/main/WordProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [16]:
import pandas as pd
import findspark
findspark.init()

In [17]:
import pyspark
from pyspark.sql import SparkSession

In [18]:
spark=SparkSession.builder.master('local[1]').appName('Step1-WordProcessing').getOrCreate()


### Word Count

In [28]:
file_path = "/content/drive/MyDrive/DataSet/news.txt"
text_file = spark.sparkContext.textFile(file_path)
print(type(text_file))
def clean_punc(x):
  punc='!"”“#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-0123456789۰۱۲۳۴۵۶۷۸۹'
  for ch in punc:
    x = x.replace(ch,'')
  x=x.replace('‌','')
  return x.strip()

text_file = text_file.map(clean_punc)
words = text_file.flatMap(lambda line: line.split(" "))
# print(words.collect())
words = words.filter(lambda x:x!='')
print(words.collect())
wordCounts_rdd = words.map(lambda  word:('TotalWords',1))
print(wordCounts_rdd.collect())
wordCounts = wordCounts_rdd.reduceByKey(lambda a,b:a+b).collect()
wordCounts


<class 'pyspark.rdd.RDD'>
['فاطمی', 'امین', 'را', 'بیشتر', 'با', 'ستاد', 'تبصره', 'میشناسند،', 'ستادی', 'که', 'در', 'اواسط', 'دهه', 'و', 'در', 'دولت', 'نهم', 'دایر', 'شد', 'و', 'اسقاط', 'و', 'جایگزینی', 'خودروهای', 'فرسوده', 'و', 'دوگانهسوز', 'کردن', 'خودروها،', 'دو', 'هدف', 'اصلی', 'آن', 'بودوی', 'ابتدا', 'مجری', 'طرح', 'نوسازی', 'خودروهای', 'فرسوده', 'کشور', 'بود', 'و', 'سپس', 'معاونت', 'برنامهریزی', 'و', 'ارزیابی', 'ستاد', 'تبصره', 'را', 'به', 'عهده', 'گرفت', 'و', 'تا', 'قائم', 'مقامی', 'این', 'ستاد', 'نیز', 'پیش', 'رفت', 'فاطمی', 'امین', 'بعدها', 'و', 'تا', 'اواخر', 'دولت', 'دهم', 'معاونت', 'برنامهریزی', 'وزارت', 'صنعت،', 'معدن', 'و', 'تجارت', 'را', 'به', 'دست', 'داشت،', 'ضمن', 'آنکه', 'برای', 'مقطعی', 'سخنگوی', 'این', 'وزارتخانه', 'نیز', 'بودبنابراین', 'سوابق', 'وی', 'نشان', 'میدهد', 'با', 'صنعت', 'و', 'به', 'ویژه', 'خودروسازی', 'غریبه', 'نیست', 'و', 'اگر', 'بتواند', 'رای', 'اعتماد', 'مجلسیها', 'را', 'بگیرد،', 'این', 'فرصت', 'را', 'خواهد', 'داشت', 'تا', 'تحولاتی', 'را', 'در', 'خود

[('TotalWords', 23151)]

In [29]:
words.take(10)

['فاطمی',
 'امین',
 'را',
 'بیشتر',
 'با',
 'ستاد',
 'تبصره',
 'میشناسند،',
 'ستادی',
 'که']

In [30]:
word_pairs = words.map(lambda  word:(word,1))
print(word_pairs.take(10))
wordCounts = word_pairs.reduceByKey(lambda a,b:a+b).sortBy(lambda a: -a[1])
results=wordCounts.take(20)
results

[('فاطمی', 1), ('امین', 1), ('را', 1), ('بیشتر', 1), ('با', 1), ('ستاد', 1), ('تبصره', 1), ('میشناسند،', 1), ('ستادی', 1), ('که', 1)]


[('و', 1035),
 ('در', 933),
 ('به', 618),
 ('از', 572),
 ('این', 524),
 ('که', 510),
 ('است', 392),
 ('را', 359),
 ('با', 307),
 ('برای', 168),
 ('کشور', 136),
 ('بر', 112),
 ('آن', 109),
 ('یک', 105),
 ('دولت', 97),
 ('خود', 95),
 ('شده', 94),
 ('هم', 82),
 ('قرار', 81),
 ('تا', 78)]

In [9]:
stop_words=wordCounts.map(lambda w : w[0]).take(20)
stop_words

['و',
 'در',
 'به',
 'از',
 'این',
 'که',
 'است',
 'را',
 'با',
 'برای',
 'کشور',
 'بر',
 'آن',
 'یک',
 'دولت',
 'خود',
 'شده',
 'هم',
 'قرار',
 'تا']

#### Bigrams

In [11]:
line="خدایا چنان کن سرانجام کار که تو خوشنود باشی و ما رستگار"
print(line)
words=[w for w in line.split(" ") if w not in stop_words and len(w.strip())>1]
print(words[:-1])
print(words[1:])
print("-"*50)
bigrams= [(w1,w2) for w1,w2 in zip(words[:-1],words[1:])]
bigrams

خدایا چنان کن سرانجام کار که تو خوشنود باشی و ما رستگار
['خدایا', 'چنان', 'کن', 'سرانجام', 'کار', 'تو', 'خوشنود', 'باشی', 'ما']
['چنان', 'کن', 'سرانجام', 'کار', 'تو', 'خوشنود', 'باشی', 'ما', 'رستگار']
--------------------------------------------------


[('خدایا', 'چنان'),
 ('چنان', 'کن'),
 ('کن', 'سرانجام'),
 ('سرانجام', 'کار'),
 ('کار', 'تو'),
 ('تو', 'خوشنود'),
 ('خوشنود', 'باشی'),
 ('باشی', 'ما'),
 ('ما', 'رستگار')]

In [31]:
def get_bigrams(line):
    words=[w for w in line.split(" ") if w not in stop_words and len(w.strip())>1]
    bigrams= [(w1,w2) for w1,w2 in zip(words[:-1],words[1:])]
    return bigrams

In [32]:
bigrams = text_file.flatMap(get_bigrams)\
            .map(lambda bigram : (bigram,1)) \
            .reduceByKey(lambda a,b : a+b) \
            .filter(lambda x : x[1] > 5)\
            .sortBy(lambda x : -x[1]) 
            

bigrams.collect()


[(('بورس', 'کالا'), 26),
 (('قیمتهای', 'پایه'), 21),
 (('نشان', 'میدهد'), 19),
 (('فاطمی', 'امین'), 18),
 (('سال', 'گذشته'), 17),
 (('خواهد', 'بود'), 15),
 (('بازار', 'سیمان'), 14),
 (('داشته', 'باشد'), 13),
 (('تامین', 'مالی'), 13),
 (('وزرای', 'پیشنهادی'), 12),
 (('میلیارد', 'دلار'), 12),
 (('حکمرانی', 'آب'), 12),
 (('وجود', 'دارد'), 11),
 (('خواهد', 'شد'), 11),
 (('وزیر', 'صمت'), 10),
 (('ایران', 'خودرو'), 10),
 (('رای', 'اعتماد'), 9),
 (('نظر', 'میرسد'), 9),
 (('رشد', 'اقتصادی'), 9),
 (('مدیریت', 'آب'), 9),
 (('سیستم', 'حکمرانی'), 9),
 (('خودروهای', 'فرسوده'), 8),
 (('وزارت', 'صنعت،'), 8),
 (('صنعت،', 'معدن'), 8),
 (('معدن', 'تجارت'), 8),
 (('واردات', 'خودرو'), 8),
 (('تحت', 'تاثیر'), 8),
 (('وجود', 'ندارد'), 8),
 (('مجلس', 'شورای'), 7),
 (('تامین', 'انرژی'), 7),
 (('قیمت', 'پایه'), 7),
 (('داشته', 'باشند'), 7),
 (('امنیت', 'آبی'), 7),
 (('صمت', 'سیزدهم'), 6),
 (('رضایت', 'مشتریان'), 6),
 (('چهار', 'ماه'), 6),
 (('مدت', 'مشابه'), 6),
 (('مشابه', 'سال'), 6),
 (('حدود', 'درصد'), 6),


In [33]:
df = pd.DataFrame(bigrams.collect())
df.columns=['Bigram','Count']
df

,Bigram,Count
0,"(بورس, کالا)",26
1,"(قیمتهای, پایه)",21
2,"(نشان, میدهد)",19
3,"(فاطمی, امین)",18
4,"(سال, گذشته)",17
5,"(خواهد, بود)",15
6,"(بازار, سیمان)",14
7,"(داشته, باشد)",13
8,"(تامین, مالی)",13
9,"(وزرای, پیشنهادی)",12


In [ ]:
spark.stop()